<a href="https://colab.research.google.com/github/rrfsantos/Desafios-NLP/blob/main/Desafio-NLP--Question%26Answer/Universal_Encoder_Q%26A_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Recuperação de perguntas e respostas utilizando codificador universal de frases multilíngue

##### https://www.tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa

Being able to automatically answer questions accurately remains a difficult problem in natural language processing. This dataset has everything you need to try your own hand at this task. Can you correctly generate the answer to questions given the Wikipedia article text the question was originally generated from?

Content:
There are three question files, one for each year of students: S08, S09, and S10, as well as 690,000 words worth of cleaned text from Wikipedia that was used to generate the questions.

The "questionanswerpairs.txt" files contain both the questions and answers. The columns in this file are as follows:

ArticleTitle is the name of the Wikipedia article from which questions and answers initially came.
Question is the question.
Answer is the answer.
DifficultyFromQuestioner is the prescribed difficulty rating for the question as given to the question-writer.
DifficultyFromAnswerer is a difficulty rating assigned by the individual who evaluated and answered the question, which may differ from the difficulty in field 4.
ArticleFile is the name of the file with the relevant article

Questions that were judged to be poor were discarded from this data set.
There are frequently multiple lines with the same question, which appear if those questions were answered by multiple individuals. https://www.kaggle.com/rtatman/questionanswer-dataset

### Solução

##### Cada resposta no dataset QA (Sxx_question_answer_pairs.txt) e seu contexto (o texto em torno da frase) em seu artigo correspondente (Sxx_setx_ax.txt.clean), são codificados em embeddings de alta dimensão com o response_encoder. Esses embeddings são armazenados em um índice construído usando a biblioteca simpleneighbors para recuperação de perguntas e respostas.

In [34]:
!python -m pip install -q tensorflow_text
!python -m pip install -q simpleneighbors[annoy]

In [76]:
#import numpy as np
import pandas as pd
import glob
import re
import os
import tqdm.notebook as tq
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer
from simpleneighbors import SimpleNeighbors
import random
import pprint
from IPython.display import HTML, display

In [36]:
from google.colab import drive
drive.mount('/content/drive')

import os
workdir_path = '/content/drive/My Drive/desafio 3/'  # Inserir o local da pasta onde estão os arquivos de entrada (treino e teste)
os.chdir(workdir_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Extrair o **contexto** dos artigos Sxx_setx_ax.txt.clean

In [37]:
files = glob.glob('/content/drive/My Drive/desafio 3/text_data/*.clean')

list_text = []
list_file = []

for file in files:
  with open(file, 'r', encoding = 'utf-8', errors='ignore') as f:
    text = f.read()
  list_text.append(re.sub(r'\n+', ' ', text).strip())
  list_file.append((os.path.basename(file)).split('.')[0])
df_context = pd.DataFrame({'Context': list_text, 'ArticleFile': list_file})

#### Criar dataframes de QA Sxx_question_answer_pairs.txt

In [92]:
df_08 = pd.read_table('S08_question_answer_pairs.txt')
df_09 = pd.read_table('S09_question_answer_pairs.txt')
df_10 = pd.read_table('S10_question_answer_pairs.txt', engine = 'python', error_bad_lines = False)

df_qa = pd.concat([df_08, df_09, df_10], axis=0, ignore_index=True)

Skipping line 765: '	' expected after '"'
Skipping line 876: '	' expected after '"'
Skipping line 1219: '	' expected after '"'


In [93]:
df_qa.drop(['DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleTitle'], axis = 1, inplace=True)
#df_09.drop(['DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleTitle'], axis = 1, inplace=True)
#df_10.drop(['DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleTitle'], axis = 1, inplace=True)

In [94]:
# Verifica se há dados faltantes na base
print('-' * 15, df_qa.isna().sum(), sep='\n')
#print('-' * 15, df_09.isna().sum(), sep='\n')
#print('-' * 15, df_10.isna().sum(), sep='\n')

---------------
Question        37
Answer         576
ArticleFile      2
dtype: int64


In [95]:
# Remove os dados faltantes da base de treino
df_qa.dropna(inplace=True)
#df_09.dropna(inplace=True)
#df_10.dropna(inplace=True)
print('-' * 15, df_qa.isna().sum(), sep='\n')
#print('-' * 15, df_09.isna().sum(), sep='\n')
#print('-' * 15, df_10.isna().sum(), sep='\n')

---------------
Question       0
Answer         0
ArticleFile    0
dtype: int64


In [96]:
df_qac = df_qa.merge(df_context, on='ArticleFile')
#df_USE09 = df_09.merge(df_context, on='ArticleFile')
#df_USE10 = df_10.merge(df_context, on='ArticleFile')

In [97]:
print('-' * 15, df_qac.shape, sep='\n')
#print('-' * 15, df_09.shape, sep='\n')
#print('-' * 15, df_10.shape, sep='\n')

---------------
(3417, 4)


In [98]:
df_qac.head()

,Question,Answer,ArticleFile,Context
0,Was Abraham Lincoln the sixteenth President of...,yes,S08_set3_a4,"Abraham Lincoln Abraham Lincoln (February 12, ..."
1,Was Abraham Lincoln the sixteenth President of...,Yes.,S08_set3_a4,"Abraham Lincoln Abraham Lincoln (February 12, ..."
2,Did Lincoln sign the National Banking Act of 1...,yes,S08_set3_a4,"Abraham Lincoln Abraham Lincoln (February 12, ..."
3,Did Lincoln sign the National Banking Act of 1...,Yes.,S08_set3_a4,"Abraham Lincoln Abraham Lincoln (February 12, ..."
4,Did his mother die of pneumonia?,no,S08_set3_a4,"Abraham Lincoln Abraham Lincoln (February 12, ..."


In [116]:
answer_context = []
question = []

for index, r in df_qac.iterrows():
  answer_context.append((r['Answer'].lower(),r['Context'].lower()))
  question.append(r['Question'].lower())

### Carregar modelo **Universal Encoder Q&A Model** do tensorflow hub

In [64]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3"
model = hub.load(module_url)

### Computar embeddings e criar o **simpleneighbors** index

In [117]:
batch_size = 100

encodings = model.signatures['response_encoder'](  input=tf.constant([answer_context[0][0]]), context=tf.constant([answer_context[0][1]]))
index = SimpleNeighbors(len(encodings['outputs'][0]), metric='angular')

print('Computing embeddings for %s sentences' % len(answer_context))
slices = zip(*(iter(answer_context),) * batch_size)
num_batches = int(len(answer_context) / batch_size)
for s in tq.tqdm_notebook(slices, total=num_batches):
  response_batch = list([r for r, c in s])
  context_batch = list([c for r, c in s])
  encodings = model.signatures['response_encoder'](
    input=tf.constant(response_batch),
    context=tf.constant(context_batch)
  )
  for batch_index, batch in enumerate(response_batch):
    index.add_one(batch, encodings['outputs'][batch_index])

index.build()
print('simpleneighbors index for %s sentences built.' % len(answer_context))


Computing embeddings for 3417 sentences



simpleneighbors index for 3417 sentences built.


### Testar o modelo

#### A questão é codificada usando **question_encoder** e o **embedding** da questão é usado para consultar o índice simpleneighbors.

In [114]:
def output_with_highlight(text, highlight):
  output = "<li> "
  i = text.find(highlight)
  while True:
    if i == -1:
      output += text
      break
    output += text[0:i]
    output += '<b>'+text[i:i+len(highlight)]+'</b>'
    text = text[i+len(highlight):]
    i = text.find(highlight)
  return output + "</li>\n"

def display_nearest_neighbors(query_text, answer_text=None):
    query_embedding = model.signatures['question_encoder'](tf.constant([query_text]))['outputs'][0]
    search_results = index.nearest(query_embedding, n=num_results)

    if answer_text:
      result_md = '''
      <p>Random Question:</p>
      <p>&nbsp;&nbsp;<b>%s</b></p>
      <p>Answer:</p>
      <p>&nbsp;&nbsp;<b>%s</b></p>
      ''' % (query_text , answer_text)
    else:
      result_md = '''
      <p>Question:</p>
      <p>&nbsp;&nbsp;<b>%s</b></p>
      ''' % query_text

    result_md += '''
      <p>Retrieved sentences :
      <ol>
    '''

    if answer_text:
      for s in search_results:
        result_md += output_with_highlight(s, answer_text)
    else:
      for s in search_results:
        result_md += '<li>' + s + '</li>\n'

    result_md += "</ol>"
    display(HTML(result_md))

In [126]:
#@title ###### Acessar os "vizinhos próximos" **nearest neighbors** para perguntas escolhidas randomicamente
num_results = 5 #@param {type:"slider", min:5, max:40, step:1}

query = random.choice(question)
display_nearest_neighbors(query)